In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

M = 4
# Create 4 single-server queues with PS scheduling
node = np.empty(4, dtype=object)
node[0] = Queue(model, 'Queue1', SchedStrategy.PS)
node[1] = Queue(model, 'Queue2', SchedStrategy.PS)
node[2] = Queue(model, 'Queue3', SchedStrategy.PS)
node[3] = Queue(model, 'Queue4', SchedStrategy.PS)  # only closed classes

source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create job classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 100, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

# Set service rates
for i in range(M):
    node[i].set_service(jobclass[0], Exp(i+1))
    node[i].set_service(jobclass[1], Exp(np.sqrt(i+1)))

# Set arrival rate for open class
source.set_arrival(jobclass[1], Exp(0.3))

In [3]:
# Setup routing matrix
M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[3], 1.0)
P.set(jobclass[0], jobclass[0], node[3], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Sink (excludes Queue4)
P.set(jobclass[1], jobclass[1], source, node[0], 1.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], sink, 1.0)

model.link(P)

In [4]:
print('This example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.')# Create solverssolver = np.array([], dtype=object)# Note: CTMC is infinite on this model due to high populationsolver = np.append(solver, SolverJMT(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=100000))solver = np.append(solver, SolverSSA(model, seed=23000, verbose=False, samples=100000))solver = np.append(solver, SolverFluid(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=100000))solver = np.append(solver, SolverMVA(model, method='lin'))solver = np.append(solver, SolverNC(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=100000))solver = np.append(solver, SolverMAM(model, cutoff=3, keep=False, verbose=True, seed=23000, samples=100000))# Solve and display resultsAvgTable = np.empty(len(solver), dtype=object)for s in range(len(solver)):    print(f'\nSOLVER: {solver[s].get_name()}')    AvgTable[s] = solver[s].get_avg_table()    print(AvgTable[s])

This example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.

SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/17764664168428740355/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 4.259317s.
  Station     JobClass     QLen    Util     RespT    ResidT    ArvR    Tput
0  Queue1  ClosedClass  98.5685  0.6977  141.7354  141.7354  0.7051  0.6980
1  Queue1    OpenClass  43.0685  0.3019  143.8462  143.8462  0.3001  0.2989
2  Queue2  ClosedClass   0.8233  0.3599    1.1341    1.1341  0.6980  0.6975
3  Queue2    OpenClass   0.4866  0.2080    1.6180    1.6180  0.2989  0.2989
4  Queue3  ClosedClass   0.3946  0.2350    0.5551    0.5551  0.6975  0.6975
5  Queue3    OpenClass   0.2862  0.1685    0.9653    0.9653  0.2989  0.2991
6  Queue4  ClosedClass   0.2169  0.1784    0.3029    0.3029  0.6975  0.6975
9  Source    OpenClass   0.0000  0.0000    0.0000    0.0000  0.0000  0.3001
  Station     JobClass     QLen    Util     RespT    ResidT   